In [4]:
from scipy.interpolate import griddata
import numpy as np
import xarray as xr

dem = pa.utils.read_data(os.path.join(data_dir, 'hgt.rdr'))
dem = xr.DataArray(dem, 
             coords = {
    "lat": (("y", "x"), lat),
    "lon": (("y", "x"), lon)
},
dims = ["y","x"])

from scipy.interpolate import griddata
import numpy as np
import xarray as xr

def interp_to_grid(u, xc, yc, new_lats, new_lons):
    new_points = np.stack(np.meshgrid(new_lats, new_lons), axis = 2).reshape((new_lats.size * new_lons.size, 2))
    print(new_points.shape)
    print(u.shape)
    print(xc.shape)
    print(yc.shape)
    z = griddata((xc, yc), u, (new_points[:,1], new_points[:,0]), method = 'nearest', fill_value = np.nan)
    out = z.reshape((new_lats.size, new_lons.size), order = "F")
    return out


values = dem.data.ravel()
lons = dem.lon.data.ravel()
lats = dem.lat.data.ravel()
_new_lats = np.linspace(dem.lat.min(), dem.lat.max(), dem.y.size)
_new_lons = np.linspace(dem.lon.min(), dem.lon.max(), dem.x.size)
new_lons = xr.DataArray(_new_lons, dims = "lon", coords = {"lon": _new_lons})
new_lats = xr.DataArray(_new_lats, dims = "lat", coords = {"lat": _new_lats})

gridded_ds = xr.apply_ufunc(interp_to_grid,
                     values, lons, lats, new_lats, new_lons,
                     input_core_dims = [[],[],[],["lat"],["lon"]],
                     output_core_dims = [['lat', 'lon']],
                     )


(109020, 2)
(109020,)
(109020,)
(109020,)


In [5]:
gridded_ds.where(~dem.isnull())

<xarray.DataArray (lat: 460, lon: 237, y: 460, x: 237)>
array([[[[ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02],
         [ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02],
         [ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02],
         ...,
         [ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02],
         [ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02],
         [ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02]],

        [[ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02],
         [ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02],
         [ 2.64135195e+02,  2.64135195e+02,  2.64135195e+02, ...,
           2.64135195e+02,  2.64135195e+02,  2.64135195e+02],
...
         [ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03],
         [ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03],
         [ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03]],

        [[ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03],
         [ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03],
         [ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03],
         ...,
         [ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03],
         [ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03],
         [ 1.51984138e+03,  1.51984138e+03,  1.51984138e+03, ...,
           1.51984138e+03,  1.51984138e+03,  1.51984138e+03]]]])
Coordinates:
  * lat      (lat) float64 31.25 31.26 31.26 31.26 ... 32.64 32.65 32.65 32.65
  * lon      (lon) float64 130.2 130.3 130.3 130.3 ... 131.2 131.2 131.3 131.3
Dimensions without coordinates: y, x

In [2]:
import pyaps3 as pa
import os
import matplotlib.pyplot as plt

# run pyaps3
data_dir = '../pyAPS_data'
# read geometry files
print('read ISCE geometry files: hgt/los/lat/lon.rdr')
dem_pa = pa.utils.read_data(os.path.join(data_dir, 'hgt.rdr'))
inc = pa.utils.read_data(os.path.join(data_dir, 'los.rdr'), dname='inc')
lat = pa.utils.read_data(os.path.join(data_dir, 'lat.rdr'))
lon = pa.utils.read_data(os.path.join(data_dir, 'lon.rdr'))

# calculate
print('calculate tropospheric delay from GRB files...')
print('------------------------------------------------')
grb_file1 = os.path.join(data_dir, 'ERA5/ERA5_N30_N40_E120_E140_20101017_14.grb')
grb_file2 = os.path.join(data_dir, 'ERA5/ERA5_N30_N40_E120_E140_20110117_14.grb')
obj1 = pa.PyAPS(grb_file1, dem=dem_pa, inc=inc, lat=lat, lon=lon, grib='ERA5', verb=True)
obj2 = pa.PyAPS(grb_file2, dem=dem_pa, inc=inc, lat=lat, lon=lon, grib='ERA5', verb=True)
phs = obj2.getdelay(wvl = 0.238403545) - obj1.getdelay(wvl = 0.238403545)

# regrid data to xarray
import numpy as np
import pandas as pd
import xarray as xr

import sys
sys.path.append('..')

from phase_o_matic.utils import regrid_twoD_coords

dem = pa.utils.read_data(os.path.join(data_dir, 'hgt.rdr'))
dem = xr.DataArray(dem, 
             coords = {
    "lat": (("y", "x"), lat),
    "lon": (("y", "x"), lon)
},
dims = ["y","x"])
dem = regrid_twoD_coords(dem)

inc = np.deg2rad(pa.utils.read_data(os.path.join(data_dir, 'los.rdr'), dname='inc'))
inc= xr.DataArray(inc, 
             coords = {
    "lat": (("y", "x"), lat),
    "lon": (("y", "x"), lon)
},
dims = ["y","x"])
inc = regrid_twoD_coords(inc)


pyaps_phase = xr.DataArray(phs, 
             coords = {
    "lat": (("y", "x"), lat),
    "lon": (("y", "x"), lon)
},
dims = ["y","x"])
pyaps_phase = regrid_twoD_coords(pyaps_phase)

dem_ds = dem.to_dataset(name = 'dem')
dem_ds['inc'] = inc
dem_ds = dem_ds.rename({'lat':'y', 'lon':'x'})

dates = (pd.to_datetime('2010-10-17T14:00'), pd.to_datetime('2011-01-17T14:00'))

# Phase o matic now
from shapely import geometry
from phase_o_matic.download import download_era
from phase_o_matic.preprocess import get_vapor_partial_pressure, convert_pressure_to_pascals, interpolate_to_heights, geopotential_to_geopotential_heights
from phase_o_matic.phase_delay import calculate_refractive_indexes, get_delay


eras = []
for date in dates:
    out_fp = download_era(date, out_dir = '../data/phaseo_era/', subset = geometry.box(129.25, 30.25, 132.25, 33.75), humid_param = 'specific_humidity')
    era = xr.open_dataset(out_fp)
    era = convert_pressure_to_pascals(era)
    era = get_vapor_partial_pressure(era)
    era = geopotential_to_geopotential_heights(era)
    if date == pd.to_datetime('2010-10-17T14:00'):
        ds = interpolate_to_heights(era, max_alt = obj1.hgt[-1])
    elif date == pd.to_datetime('2011-01-17T14:00'):
        ds = interpolate_to_heights(era, max_alt = obj2.hgt[-1])
    tmp = era.copy()
    ds = calculate_refractive_indexes(ds)
    ds = get_delay(ds, dem_ds, wavelength = 0.238403545)
    eras.append(ds['delay'])
era = xr.concat(eras, dim = 'time')

# now plot the two
fig, axes = plt.subplots(2, 2, figsize = (12, 8))

titles = ['PyAPS (mintpy) Tropospheric delay', 'Height']
for ax, data, title in zip(axes[0, :], [pyaps_phase.data, dem_ds.dem.data], titles):
    if title == 'Height':
        im = ax.imshow(data, interpolation='nearest', vmin = 0, vmax = 1650, aspect="auto")
    else:
        im = ax.imshow(data, interpolation='nearest', aspect="auto", vmin = -4.5, vmax = -0.4)
    # axis format
    ax.invert_yaxis()
    ax.set_title(title)
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('meter')

phaseo = (era.isel(time = 1) - era.isel(time = 0))
phaseo.plot(ax = axes[1, 0], vmin = -4.5, vmax = -0.4)
(pyaps_phase - phaseo).plot(ax = axes[1, 1], vmax = 1, vmin = -1)
axes[1,0].set_title('Phase-o-matic Tropospheric Delay')
axes[1,1].set_title('Difference')

plt.savefig('../images/pyaps_phaseo_compare.png')

read ISCE geometry files: hgt/los/lat/lon.rdr
calculate tropospheric delay from GRB files...
------------------------------------------------
INFO: INCIDENCE ANGLE AS AN ARRAY
INFO: AREA COVERAGE IN SNWE: (33.85, 30.05, 129.05, 132.45)
PROGRESS: READING GRIB FILE
INFO: USING PRESSURE LEVELS OF ERA-INT OR ERA-5 DATA
INFO: IMAGE DIMENSIONS: 15 LATITUDES AND 13 LONGITUDES


KeyboardInterrupt: 

In [75]:
dataset = tmp
max_alt = None
n_heights = 300
min_alt = -200
max_alt = dataset['gph'].max().round() if not max_alt else max_alt
heights = np.linspace(min_alt, max_alt, n_heights)

In [ ]:
gridded_ds = xr.apply_ufunc(interp_to_grid,
                     values, lons, lats, new_lats, new_lons,
                     vectorize = True,
                     dask = "parallelized",
                     input_core_dims = [['site_id'],['site_id'],['site_id'],["lat"],["lon"]],
                     output_core_dims = [['lat', 'lon']],
                     )


In [76]:
dataset.interp(heights = heights)

ValueError: Dimensions {'heights'} do not exist. Expected one or more of Frozen({'longitude': 13, 'latitude': 15, 'level': 37, 'time': 1})